# TP2 - Sistemas Autónomos
#### "Monitorização de parâmetros ambientais para a prática de desporto"

## Import Libraries

In [ ]:
import sys
import config
import firebase_admin
from firebase_admin import auth, credentials, firestore
import numpy as np
import pandas as pd
import seaborn as sns
import re
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt

## Get the Data


### Get the Database from Cloud Firestore

In [ ]:
# Get the key
cred = credentials.Certificate(config.firestore_key)

# Initialize/Get the app
try:
    app = firebase_admin.initialize_app(cred)
except:
    app = firebase_admin.get_app()

# Get the database
db = firestore.client()

### Get the 'WM', 'UV' and 'AQ' collections

In [ ]:
pd.set_option('display.max_columns', None)

# ------- Open Weather Map -------

dataWM = pd.DataFrame(columns=['year','month','day','hours','minutes','hours.minutes','feels_like','general_weather','humidity','pressure','temp_min','temp_max','wind_speed'])

wm_ref = db.collection(u'WM')
docsWM = wm_ref.stream()

# general_weather tem que se meter em integer, mas temos que decidir se continuamos a extrair o "description" ou o "main" no momento do request

for doc in docsWM:
    params = doc.to_dict()
    datetime = re.split(r'[-T:]',doc.id)
    perc_minutes = int(datetime[3])+(0.0167*int(datetime[4])) # precisava de ter os minutos adaptados para a unidade da hora
    dataWM = dataWM.append({'year':int(datetime[0]),'month':int(datetime[1]) ,'day':int(datetime[2]), 'hours':int(datetime[3]), 'minutes': int(datetime[4]), 'hours.minutes': perc_minutes, 'feels_like': float(params['feels_like'])-273.15, 'general_weather':params['general_weather'], 'humidity': int(params['humidity']), 'pressure': int(params['pressure']), 'temp_min': float(params['temp_min'])-273.15, 'temp_max': float(params['temp_max'])-273.15, 'wind_speed': float(params['wind_speed'])}, ignore_index=True)


# ------- Open UV -------

dataUV = pd.DataFrame(columns=['uv','uv_time','uv_max','uv_max_time','st1','st2','st3','st4','st5','st6'])

uv_ref = db.collection(u'UV')
docsUV = uv_ref.stream()

for doc in docsUV:
    params = doc.to_dict()
    dataUV = dataUV.append({'uv': float(params['uv']), 'uv_time': params['uv_time'], 'uv_max': float(params['uv_max']), 'uv_max_time': params['uv_max_time'], 'st1': params['st1'], 'st2': params['st2'], 'st3': params['st3'], 'st4': params['st4'], 'st5': params['st5'], 'st6': params['st6']}, ignore_index=True)


# ------- Open AQ -------

dataAQ = pd.DataFrame(columns=['no2','o3','pm10'])

aq_ref = db.collection(u'AQ')
docsAQ = aq_ref.stream()

for doc in docsAQ:
    params = doc.to_dict()
    dataAQ = dataAQ.append({'no2': int(params['no2']), 'o3': int(params['o3']), 'pm10': int(params['pm10'])}, ignore_index=True)


data = pd.concat([dataWM, dataUV, dataAQ], axis=1)
data.head()


In [ ]:
data.info()

## Exploratory Data Analysis

In [ ]:
print(data.isnull().sum(axis=0))
sns.heatmap(data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
sns.set_palette("GnBu_d")
sns.set_style('whitegrid')
#sns.jointplot(x='hours.minutes', y='feels_like', data=data)
#sns.jointplot(x='hours.minutes', y='feels_like', data=data, kind='kde')
sns.jointplot(x='hours.minutes', y='feels_like', data=data, kind='hex')


## LSTM - Prediction

In [ ]:
def prepare_data(df):
    feels_like_history = data[['feels_like']]
    return feels_like_history


def normalize_data(df):
    scaler = MinMaxScaler(feature_range=(0, 1))
    df = scaler.fit_transform(df)
    return scaler


def plotDataset(data, predictions):
    plt.figure(figsize=(8, 6))
    plt.plot(range(len(data)), data['feels_like'], color='blue', label='Confirmed')
    plt.plot(range(len(data)-1, len(data)+len(predictions)-1),
             predictions, color='red', label='Prediction')
    plt.title('Weather')
    plt.ylim(0,30) 
    plt.ylabel('Feels_like')
    plt.xlabel('Iterations')
    plt.show()


def to_supervised(df, timesteps):
    data = df.values  # array de arrays com os valores
    X, y = list(), list()
    dataset_size = len(data)  # nr de linhas
    for curr_pos in range(dataset_size):
        input_index = curr_pos+timesteps
        label_index = input_index+1
        if label_index < dataset_size:
            X.append(data[curr_pos:input_index, :])
            y.append(data[input_index:label_index, 0])
    return np.array(X), np.array(y)

def rmse(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

def build_model(timesteps, features, dropout_rate=0.5):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(64, return_sequences=True,
                                   input_shape=(timesteps, features)))
    model.add(tf.keras.layers.LSTM(
        128, return_sequences=True, dropout=dropout_rate))
    model.add(tf.keras.layers.LSTM(
        256, return_sequences=False, dropout=dropout_rate))
    model.add(tf.keras.layers.Dense(64, activation='tanh'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(features, activation='linear'))
    model.compile(
        loss=rmse,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy','mae',rmse])
    print(model.summary())
    return model

def forecast(model, df, timesteps, multisteps, scaler):
    input_seq = df[-timesteps:].values
    inp = input_seq
    predictions = list()
    for step in range(1, multisteps+1):
        inp = inp.reshape(1, timesteps, 1)
        yhat = model.predict(inp, verbose=1)
        yhat_inversed = scaler.inverse_transform(yhat)
        predictions.append(yhat_inversed[0][0])
        inp = np.append(inp[0], yhat)
        inp = inp[-timesteps:]
    return predictions

In [ ]:
timesteps = 5
multistep = 7
features = 1
batch_size = 16

df_prepared = prepare_data(data)
print(df_prepared)
df_to_plot = df_prepared.copy()

scaler = normalize_data(df_prepared)
X, y = to_supervised(df_prepared, timesteps)
model = build_model(timesteps, features)
model.fit(X, y, shuffle=False, epochs=50, verbose=1, batch_size=batch_size)

predictions = forecast(model, df_prepared, timesteps, multistep, scaler)
plotDataset(df_to_plot, predictions)
print(predictions)